In [3]:
# Import packages and initiate engine
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

In [4]:
# Open the connection
with engine.connect() as con:
    Bookings = pd.read_sql_query("SELECT * FROM Bookings", engine)
    Facilities = pd.read_sql_query("SELECT * FROM Facilities", engine)
    Members = pd.read_sql_query("SELECT * FROM Members", engine)

In [5]:
Bookings.head()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


In [6]:
Facilities.head()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


In [7]:
Members.head()

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05


In [8]:
# Answer Case Study PART 2 Questions
# Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue.
Bookings_Facilities = Bookings.merge(Facilities, on = 'facid', how = 'left')
Bookings_Facilities.head()

,bookid,facid,memid,starttime,slots,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,3,1,2012-07-03 11:00:00,2,Table Tennis,0.0,5.0,320,10
1,1,4,1,2012-07-03 08:00:00,2,Massage Room 1,9.9,80.0,4000,3000
2,2,6,0,2012-07-03 18:00:00,2,Squash Court,3.5,17.5,5000,80
3,3,7,1,2012-07-03 19:00:00,2,Snooker Table,0.0,5.0,450,15
4,4,8,1,2012-07-03 10:00:00,1,Pool Table,0.0,5.0,400,15


In [12]:
# Calculate revenue for all bookings
Bookings_Facilities['revenue'] = 0
guestbooking = Bookings_Facilities['memid'] == 0
memberbooking = Bookings_Facilities['memid'] != 0
Bookings_Facilities.loc[guestbooking, 'revenue'] = Bookings_Facilities['guestcost']*Bookings_Facilities['slots']
Bookings_Facilities.loc[memberbooking, 'revenue'] = Bookings_Facilities['membercost']*Bookings_Facilities['slots']

In [18]:
# Group the dataframe by facility name
result = Bookings_Facilities.groupby(['facid','name'])['revenue'].sum().reset_index()
result = result[result['revenue'] < 1000 ]

In [19]:
result.sort_values('revenue')
print(result)

   facid           name  revenue
3      3   Table Tennis    180.0
7      7  Snooker Table    240.0
8      8     Pool Table    270.0
